# MSEE UQ short course:  The $\texttt{UQpy}$ library

Application of surrogate modeling using the $\texttt{UQpy}$ module $\texttt{surrogates}$.

Detailed instructions on how to use this module can be found in the $\texttt{UQpy}$ documentation.

https://uqpyproject.readthedocs.io/en/latest/surrogates/index.html

# Activity 1

1. Build the PCE surrogate for the 1-D Bouc-Wen model for:
    - For 100 training data.
    - maximum PCE order p=4.
    - Ridge regression methods.
    For this case, estimate the first two moments (mean and variance) of the PCE surrogate.


In [1]:
# Activity 1 - Solution
from UQpy.run_model.RunModel import RunModel
from UQpy.run_model.model_execution.PythonModel import PythonModel
import numpy as np
import matplotlib.pyplot as plt
from UQpy.surrogates import *
from UQpy.sampling import MonteCarloSampling
from UQpy.distributions.collection import Uniform

distribution=Uniform(0.5, 3.0)


training_sampling = MonteCarloSampling(distributions=distribution, nsamples=100)
training_samples=training_sampling.samples

m=PythonModel(model_script='model_1D.py', model_object_name='boucwen_runmodel', var_names=['r0'])
boucwen = RunModel(m)
boucwen.run(samples=training_samples)

maximum_displacement = boucwen.qoi_list[:100]

from UQpy.surrogates.polynomial_chaos import PolynomialBasis, RidgeRegression, PolynomialChaosExpansion

polys = TotalDegreeBasis(distributions=distribution, max_degree=4)
ridge = RidgeRegression()
pce = PolynomialChaosExpansion(polynomial_basis=polys,regression_method=ridge)

pce.fit(training_samples,np.array(maximum_displacement).reshape(100,1))

prediction_sampling=MonteCarloSampling(distributions=[distribution], nsamples=100)
prediction_results=pce.predict(prediction_sampling.samples)

print('Moments from Ridge regression :', pce.get_moments())

Moments from Ridge regression : (6.0449, 0.9782)


## Activity 2

1. Build the Gaussian Process Regression surrogate for the 1-D contact sphere model for:
    - For 100 training data.
    - Quadratic regression model
    - Matern kernel


In [ ]:
K1 = GaussianProcessRegression(regression_model=QuadraticRegression(),
                               kernel=Matern(), optimizations_number=20, 
                               hyperparameters=[1, 0.1])

K1.fit(samples=training_samples.reshape(-1,1), values=maximum_displacement)


# Activity 3


## Replace the $\texttt{ThirdParty}$ model that calculates the Hugoniot relationships with a $\texttt{GaussianProcessRegression}$.

The model consists of the Rankine-Hugoniot equations. These equations describe the relationship between the states on both sides of a shock wave and express the conservation of mass, momentum and energy:

   \begin{align*}
    & \rho_0 U_s = \rho_1(U_s -u_p)\\
    & P_1=\rho_0 U_s u_p \\
    & E_1 -E_0=\frac{1}{2}(P_1+P_0)(\frac{1}{\rho_0}-\frac{1}{\rho_1})
   \end{align*}

given the relationship between $U_s, u_p$ can be computed and subsequently the quantities $\rho_1, P_1$ and $E_1$ can be computed. We know that the relationship between the shock velocity $U_s$ and particle velocity $u_p$ is given by a cubic polynomial expression as follows:

   \begin{align*}
    U_s = a_0 + a_1 \cdot u_p + a_2 \cdot u_p^2+ a_3 \cdot u_p^3
   \end{align*}
   
where the polynomial parameters are uniformly distributed in the following ranges:

   \begin{align*}
    & \texttt{a}\_\texttt{0} \sim \mathcal{U}(5.3, 7.9) \\
    & \texttt{a}\_\texttt{1} \sim \mathcal{U}(0.83, 1.12) \\
    & \texttt{a}\_\texttt{2} \sim \mathcal{U}(-0.014, -0.002) \\
    & \texttt{a}\_\texttt{3} \sim \mathcal{U}(3.68e-4, 4.59e-4)
   \end{align*}

To create the input data for the $\texttt{GaussianProcessRegression}$ the RunModel of the $\texttt{Day1}$ can be used. The surrogate given the a particle velocity values will be able to return all other quantities $U_s, \rho_1, P_1, E_1$ at the specific state of the material.

- The user must create a $\texttt{GaussianProcessRegression}$ for each one of the following mappings:
    - Particle Velocity $u_p$ - Shock Velocity $U_s$
    - Particle Velocity $u_p$ - Density $\rho_1$
    - Particle Velocity $u_p$ - Pressure $P_1$
    - Particle Velocity $u_p$ - Energy $E_1$
    
Then given a set of $50$ equidistant points in the range $[0,10]$ predict the respective quantities and plot their evolution over particle velocity. 

In [2]:
#Solution
import numpy as np
import matplotlib.pyplot as plt
from UQpy.run_model.RunModel import RunModel
from UQpy.run_model.model_execution.ThirdPartyModel import ThirdPartyModel
from UQpy.distributions import Uniform
from UQpy.sampling.MonteCarloSampling import MonteCarloSampling

dist1 = Uniform(loc=5.3, scale=2.6)
dist2 = Uniform(loc=0.83, scale=0.29)
dist3 = Uniform(loc=2.7, scale=1.2)
dist4 = Uniform(loc=3.68e-4, scale=4.59e-4)

x = MonteCarloSampling(distributions=[dist1, dist2, dist3, dist4], nsamples=5)
samples = x.samples

m1=ThirdPartyModel(model_script='HugoniotThirdPartyRunner.py', model_object_name = 'run_model',
                  input_template='HugoniotCalculations.py', var_names=['a0', 'a1', 'a2', 'a3'],
                  output_script='process_hugoniot_output.py', output_object_name='read_output')
model_serial_third_party=RunModel(samples=samples,  model=m1)

In [3]:
up=[]
us=[]
d=[]
p=[]
e=[]
for result in model_serial_third_party.qoi_list:
    up.extend(result[0])
    us.extend(result[1])
    d.extend(result[2])
    p.extend(result[3])
    e.extend(result[4])

In [4]:
from UQpy.surrogates.gaussian_process import *

K_Us = GaussianProcessRegression(kernel=RBF(), optimizations_number=20, 
                                 hyperparameters=[1, 0.1, 0.01], noise=True)
K_D = GaussianProcessRegression(kernel=RBF(), optimizations_number=20, 
                                 hyperparameters=[1, 0.1, 0.01], noise=True)
K_P = GaussianProcessRegression(kernel=RBF(), optimizations_number=20, 
                                 hyperparameters=[1, 0.1, 0.01], noise=True)
K_E = GaussianProcessRegression(kernel=RBF(), optimizations_number=20, 
                                 hyperparameters=[1, 0.1, 0.01], noise=True)

K_Us.fit(samples=np.array(up).reshape(-1,1), values=np.array(us).reshape(-1,1))
K_D.fit(samples=np.array(up).reshape(-1,1), values=np.array(d).reshape(-1,1))
K_P.fit(samples=np.array(up).reshape(-1,1), values=np.array(p).reshape(-1,1))
K_E.fit(samples=np.array(up).reshape(-1,1), values=np.array(e).reshape(-1,1))

In [ ]:
prediction_particle_velocities = np.linspace(3.5, 8, 100)

predicted_shock_velocity=[]
predicted_density=[]
predicted_pressure=[]
predicted_energy=[]

for x in prediction_particle_velocities:
    predicted_shock_velocity.extend(K_Us.predict(x))
    predicted_density.extend(K_D.predict(x))
    predicted_pressure.extend(K_P.predict(x))
    predicted_energy.extend(K_E.predict(x))